In [4]:
cd yolov5/

/mnt/opt/alexw/Work/ITMO/Dl_practice/yolov5


In [6]:
import argparse
import csv
import os
import platform
import sys
from pathlib import Path
from tqdm import tqdm
import torch


from ultralytics.utils.plotting import Annotator, colors, save_one_box

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.torch_utils import select_device, smart_inference_mode


In [17]:
# Load model
weights='runs/train/exp3/weights/best.pt'
#
imgsz=(640, 640)
device = select_device('cpu')
data='dataset/dataset.yaml'
model = DetectMultiBackend(weights, device=device, dnn=False, data=data, fp16=False,  )
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # check image size


YOLOv5 🚀 v7.0-245-g3d8f004 Python-3.8.10 torch-2.1.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs


In [36]:
import torch
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from utils.general import non_max_suppression, scale_boxes
from utils.plots import Annotator

def predict_and_draw_bboxes(model, image: Image, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic_nms=False, max_det=1000):
    device = "cpu"#torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Convert PIL image to tensor
    image_tensor = transforms.ToTensor()(image).unsqueeze(0).to(device)

    # Inference
    preds = model(image_tensor)

    # Apply NMS
    preds = non_max_suppression(preds, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

    # Calculate gain tensor
    gain = torch.tensor(image.size * 2).to(device)

    # Draw bounding boxes on the original image
    draw = ImageDraw.Draw(image)
    for i, det in enumerate(preds):  # detections per image
        if len(det):
            # Rescale boxes from img size to image size
            det[:, :4] = scale_boxes(gain, det[:, :4])

            # Write results
            for *xyxy, conf, cls in reversed(det):
                label = f'{int(cls)} {conf:.2f}'
                draw.rectangle(xyxy, outline="red")  # Draw the rectangle
                draw.text(xyxy[0], f"{label}", fill="red")  # Draw the label

    return image


In [37]:
image = Image.open('vyezd-s-prilegayuschei-territorii-dorozhnye-znaki-0025766172-preview.jpg')

In [38]:
predict_and_draw_bboxes(model, image)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 28 but got size 27 for tensor number 1 in the list.